In [ ]:
from sklearn.externals import joblib

In [ ]:
import MeCab
t = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ko-dic')

#from konlpy.tag import Twitter
#from konlpy.utils import pprint
#import nltk
#from nltk import word_tokenize, pos_tag
#import regex

stopwords = open('./stopwords.txt').read().split("\n")

def tokenize_mecab(text):
    
    if type(text) == type(unicode()):
        node = t.parseToNode(text.encode("utf8"))
    else:
        node = t.parseToNode(text)
    words = []
    while node:
        feature = node.feature.split(',')
        
        if feature[0].startswith('NN') or feature[0].startswith('SL'): 
            words.append(node.surface)
        node = node.next

    words = [word for word in words if not (word in stopwords)]
    return words

#twitter=Twitter()   

#def tokenize(text):
#    words=[]
#    words = twitter.nouns(text)
#    tags_en = nltk.pos_tag(regex.findall(ur'[a-zA-Z]+', text))
#    nouns = [token for token, pos in tags_en if pos.startswith('N')]
#    words.extend(nouns)
#    return words

clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')


In [ ]:
joblib.dump(clf,'n_classify.model')

In [ ]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

In [ ]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [ ]:
pred = clf.predict(vectorizer.transform(['[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)']))[0]
print cate_id_name_dict[pred]

In [ ]:
from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    pred = clf.predict(vectorizer.transform([name]))[0]
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능